In [1]:
import numpy as np
import pandas as pd

In [58]:
items = pd.read_csv('../data/items.csv')
#stores = pd.read_csv('../data/stores.csv')
#stores.city.unique()

In [59]:
items.head()

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


In [71]:
print("There are {n_items} items which fall into {n_families} families.".format(n_items = len(items),
                                                                        n_families = len(items.family.unique())))
print("The family names are:\n {families}".format(families = items.family.unique()))

TypeError: object of type 'DataFrame' has no len()

# Let's try something with Spark real quick

In [ ]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Test Comp") \
    .config("spark.driver.memory","4g")  \
    .getOrCreate()

train=spark.read.csv("../data/train.csv", inferSchema=True, header=True).cache()
items=spark.read.csv("../data/items.csv", inferSchema=True, header=True).cache()
train.createOrReplaceTempView("train")
items.createOrReplaceTempView("items")

#returns a spark dataframe
sdf = spark.sql("""SELECT  
                    train.*,
                    items.family, items.class, items.perishable
             FROM train
             LEFT JOIN items
             ON train.item_nbr = items.item_nbr
             """
         )

In [128]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Test Comp") \
    .config("spark.driver.memory","4g")  \
    .getOrCreate()

train_items=spark.read.csv("../data/fake/train_items.csv", inferSchema=True, header=True).cache()
stores=spark.read.csv("../data/fake/stores.csv", inferSchema=True, header=True).cache()
train_items.createOrReplaceTempView("train_items")
stores.createOrReplaceTempView("stores")

#returns a spark dataframe
sdf = spark.sql("""SELECT  
                    train_items.*,
                    stores.city, stores.state, stores.type, stores.cluster
             FROM train_items
             LEFT JOIN stores
             ON train_items.store_nbr = stores.store_nbr
             """
         )

In [129]:
sdf.show()

+---+------------------+---------+--------+----------+-----------+------------+-----+----------+-----+---------+----+-------+
| id|              date|store_nbr|item_nbr|unit_sales|onpromotion|      family|class|perishable| city|    state|type|cluster|
+---+------------------+---------+--------+----------+-----------+------------+-----+----------+-----+---------+----+-------+
|  0|2012-12-3123:00:00|        1|  103665|       7.0|       None|BREAD/BAKERY| 2712|         1|Quito|Pichincha|   D|     13|
|  3|2012-12-3123:00:00|        1|  108079|       1.0|       None|    GROCERYI| 1030|         0|Quito|Pichincha|   D|     13|
|  2|2012-12-3123:00:00|        3|  105575|       2.0|       None|    GROCERYI| 1045|         0|Quito|Pichincha|   D|      8|
|  4|2012-12-3123:00:00|        9|  108701|       1.0|       None|        DELI| 2644|         1|Quito|Pichincha|   B|      6|
|  1|2012-12-3123:00:00|        2|  105574|       1.0|       None|    GROCERYI| 1045|         0|Quito|Pichincha|   D| 

In [107]:
data.write.parquet("../data/output/train-joined-with-items.parquet", mode='overwrite')


In [108]:
df_from_parquet = pd.read_parquet('../data/output/train-joined-with-items.parquet', engine='pyarrow')

In [117]:
df = df_from_parquet

In [118]:
df.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable
0,0,2012-12-31 23:00:00,25,103665,7.0,None,BREAD/BAKERY,2712,1
1,1,2012-12-31 23:00:00,25,105574,1.0,None,GROCERY I,1045,0
2,2,2012-12-31 23:00:00,25,105575,2.0,None,GROCERY I,1045,0
3,3,2012-12-31 23:00:00,25,108079,1.0,None,GROCERY I,1030,0
4,4,2012-12-31 23:00:00,25,108701,1.0,None,DELI,2644,1


In [120]:
df.ix[df['unit_sales'].idxmin()]

id                        49592112
date           2015-06-21 22:00:00
store_nbr                       18
item_nbr                   1166474
unit_sales                  -15372
onpromotion                  False
family                   BEVERAGES
class                         1120
perishable                       0
Name: 49592112, dtype: object

In [76]:
data.write.parquet("../data/output/train-joined-with-items.parquet")

AnalysisException: 'Duplicate column(s): "item_nbr" found, cannot save to file.;'

In [132]:
stores = spark.read.csv("../data/stores.csv", inferSchema=True, header=True).cache()
transactions = spark.read.csv("../data/transactions.csv", inferSchema=True, header=True).cache()
stores.createOrReplaceTempView("stores")
transactions.createOrReplaceTempView("transactions")


In [133]:
stores_transactions = spark.sql("""SELECT
                    stores.*,
                    transactions.date, transactions.transactions
                    FROM stores
                    LEFT JOIN transactions
                    ON stores.store_nbr = transactions.store_nbr
                """)


In [134]:
stores_transactions.show()

+---------+--------+--------+----+-------+-------------------+------------+
|store_nbr|    city|   state|type|cluster|               date|transactions|
+---------+--------+--------+----+-------+-------------------+------------+
|       31|Babahoyo|Los Rios|   B|     10|2013-01-02 00:00:00|        1401|
|       31|Babahoyo|Los Rios|   B|     10|2013-01-03 00:00:00|        1126|
|       31|Babahoyo|Los Rios|   B|     10|2013-01-04 00:00:00|        1246|
|       31|Babahoyo|Los Rios|   B|     10|2013-01-05 00:00:00|        1317|
|       31|Babahoyo|Los Rios|   B|     10|2013-01-06 00:00:00|        1468|
|       31|Babahoyo|Los Rios|   B|     10|2013-01-07 00:00:00|        1170|
|       31|Babahoyo|Los Rios|   B|     10|2013-01-08 00:00:00|        1200|
|       31|Babahoyo|Los Rios|   B|     10|2013-01-09 00:00:00|        1007|
|       31|Babahoyo|Los Rios|   B|     10|2013-01-10 00:00:00|        1001|
|       31|Babahoyo|Los Rios|   B|     10|2013-01-11 00:00:00|        1011|
|       31|B